Introduction to NLP course (2017-2018).

Notebook 5: Chunking. Syntactic parsing.

by Venelin Kovatchev, University of Barcelona

In [1]:
# Import section

# Import NLTK
import nltk

# Import corps for chunking
from nltk.corpus import conll2000

(Why) Text structure:

- “The meaning of a complex expression is determined by the meanings of its constituent expressions and **the rules used to combine them**”
    - Formal sentence structure, no semantics
- “A dog bites a man” vs “A man bites a dog” – in a bag-of-words, these are the same
- Compositional and non-compositional elements: “A dog bites John Smith”

Parsing and generation:
- “A Dog bites a man” -> who did what to whom (is the sentence possible at all)
- “Dog”, “bite”, “man” -> what kind of sentences can be generated using these words

Shallow parsing (chunking):
- Linear I-O-B structure. 
- No hierarchy. No (complex) long distance dependencies.
- Identify constituents.
- More computationally effective and higher accuracy.
- Can use a familiar approach (HMM)

In [ ]:
# Split the conll corpus into test and train
test_sents = conll2000.chunked_sents('test.txt')
train_sents = conll2000.chunked_sents('train.txt')

# See the format of the chunked corpus
# Print the original format
print("Chunked sentence: {}".format(train_sents[99]))
# Draw a tree
train_sents[99].draw()
# Print the I-O-B format
print("Chunked sentence in I-O-B format: {}".format(nltk.chunk.tree2conlltags(train_sents[99])))

The main principle of (simplified) chunking:
- ignore the actual word and only work with the POS tag
- define (or train) a model that maps from sequences of POS tags to sequences of IOB tags
 
Regex chunking:
- POS tags and Chunk tags are relatively small closed sets
- List of manually created (regex) rules that map from one to the other

HMM chunking:
- Train an HMM model to map from POS tag sequences to IOB tag sequences

In [ ]:
# Regex chunking
# Test sentence, already POS tagged
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]

# A simple grammar to identify noun phrases
grammar = "NP: {<DT>?<JJ>*<NN>}"

# Parse the sentence
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)

print ("The result of parsing the test sentence with the manual grammar: {}".format(result))
print ("The result of parsing the test sentence with the manual grammar (IOB): {}".format(nltk.chunk.tree2conlltags(result)))

In [ ]:
# HMM chunker
class unigram_chunker(nltk.ChunkParserI):
    
    # Initialize and train the chunker
    def __init__(self, train_sents):
        # Take the pos and the iob tags of the corpus
        # Ignore the actual words, we map from pos tag to iob tag
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        # Train an unigram tagger from the train data
        self.tagger = nltk.UnigramTagger(train_data)
    
    # Parse function
    # Takes a corpus in POS tagged format
    def parse(self,sentence):
        # Take the pos tags
        pos_tags = [pos for (word,pos) in sentence]
        # Use the tagger to tag the modified corpus
        tagged_pos_tags = self.tagger.tag(pos_tags)
        # Take the chunks from the tagged corpus
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        # Convert the output
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        
        # Return the tagged sentence
        return nltk.chunk.conlltags2tree(conlltags)

uni_chunker = unigram_chunker(train_sents)
print ("The performance of unigram chunker is: {}".format(uni_chunker.evaluate(test_sents)))

(Full) Syntactic parsing:
- Assign a (hierarchical binary tree) structure to a sentence, given a grammar
- Input: Grammar (pre-defined), sentence
- Output: all possible trees (if any)

Syntactic ambiguity (multiple parses for the same sentence)
- Disambiguation through probabilistic grammers
- Rule based (heuristic) disambiguation
- Use of external resources (such as dictionaries, knowledge bases, and parsed corpora)

Context free grammar:
- sets of rules each of which expresses the ways the symbols of the language can be grouped and ordered together 
- a lexicon of words and symbols


- terminal nodes – the lexicon of the language (actual words)
- non-terminal nodes – generalization nodes (classes, such as POS)
- start symbol (S)
- derivation – a sequence of rule expansion (left to right)

In [ ]:
# A simple CFG
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

# Test sentence
sent = "Mary saw Bob".split()

# Parse the sentence using the grammar
rd_parser = nltk.RecursiveDescentParser(grammar1)

# Print all possible trees
for tree in rd_parser.parse(sent):
    print(tree)

# Draw all possible trees
for tree in rd_parser.parse(sent):
    tree.draw()


In [ ]:
# Observe the workings of the different parser algorithms

# Top-down parser
# - Start from “s”
# - Generate a tree
# - Map the tree to the terminal nodes

nltk.app.rdparser()

In [ ]:
# Bottom-up parser
# - Start from the terminal nodes.
# - Group them into phrases.
# - Try to build up until S.
nltk.app.srparser()